## Packet Class

In [57]:
class Packet:
    def __init__(self,name,arrival_time,length):
        self.name = name
        self.arrival_time = arrival_time
        self.length = length
        self.weighted_length = 0 
        self.virtual_finish_time = float('inf')
        self.actual_finish_time = float('inf')
        self.order = -1

    def set_weighted_length(self,weight):
        try:
            assert self.length % weight == 0 , "Weighted length cannot be a float number"
        except AssertionError as e:
            print(e)
            exit(1)
        self.weighted_length =  self.length // weight
        
    def set_virtual_finish_time(self,previous_finish_time):
        self.virtual_finish_time = previous_finish_time + self.weighted_length

    def set_actual_finish_time(self,previous_finish_time):
        self.actual_finish_time = previous_finish_time + self.length

    def set_order(self,order):
        self.order = order 

    def print_packet(self):
        print(f" {self.name} | {self.arrival_time} | {self.length} | {self.weighted_length} | {self.virtual_finish_time} | {self.actual_finish_time} | {self.order}")

    def __str__(self):
        return f"({self.name} , {self.arrival_time} , {self.length} )"

# Global Functions

In [58]:
def register_packets():
    # needs memoization for global packet naming , a cache that sees if the name has been named before or not 
    Packets = []
    packet_info = ""
    previous_packet_names = {} # local cache for queue packet names , it's not ok to have two packets with same name in same queue 
    # but on a global basis , it's ok to have two packets with same name in different queues
    i = 0
    while True:
        i += 1
        packet_info = input(f"> Entry {i} : ")
        if packet_info == "-1":
            del i 
            break
        else:
            packet_info = packet_info.split(" | ")
            try:
                if previous_packet_names.get(packet_info[0],None) is None:
                    previous_packet_names[packet_info[0]] = 1
                else:
                    raise ValueError("Duplicate Packet Entered ...")
                name = packet_info[0]
                arrival_time = int(packet_info[1])
                length = int(packet_info[2])
                Packets.append(Packet(name,arrival_time,length))
            except ValueError:
                print("Error in Packet format")
                del i 
                return None
            except IndexError:
                print("Missing Argument")
                del i 
                return None
    return Packets

## PacketQueue Class

In [59]:
class PacketQueue:
    id = 0

    @classmethod
    def reset_id(cls):
        cls.id = 0
    
    @classmethod
    def rename_queue(cls):
        cls.id += 1
        return "Q" + str(cls.id)
        
    def __init__(self,weight,packets):
        self.name = PacketQueue.rename_queue()
        self.weight = weight
        if packets is not None:
            self.packets = sorted(packets , key = lambda x : (x.arrival_time , ord(x.name)))
            for packet in self.packets:
                packet.set_weighted_length(weight)
        else:
            self.packets = None


    def calculate_virtual_finish_time(self):
        self.virtual_finish_times = []
        global_finish_time = 0
        if self.packets is None:
            print("No Packets in Queue")
            return self.virtual_finish_times
        for packet in self.packets:
            if packet.arrival_time >= global_finish_time : 
                packet.set_virtual_finish_time(packet.arrival_time)
            else:
                packet.set_virtual_finish_time(global_finish_time)
            global_finish_time = packet.virtual_finish_time
            self.virtual_finish_times.append((global_finish_time,PacketQueue.id,packet.name))
        return self.virtual_finish_times

    def get_packet(self,packet_name):
        return list(filter(lambda p : p.name == packet_name , self.packets))[0]
        
    
    def print_queue(self):
        print(f"-------------{self.name}--------------")
        for packet in self.packets:
            packet.print_packet()

    def __str__(self):
        return f"( {self.name} , {self.weight} , {self.packets} )"

## WFQ class

In [60]:
class WFQ:
    def __init__(self,num_of_queues):
        PacketQueue.reset_id()
        print("Welcome to WFQ registry ...")
        print("Please enter required Packet to the format given below")
        print("Format: Packet_Name | Arrival Time | Length (-1 to exit)")
        self.pqueues = []
        self.queues_vlist = []
        for i in range(num_of_queues):
            print(f"Registry of Queue {i+1} packets")
            q_packets = register_packets()
            if q_packets is None:
                print("Error in Packet Registry , Discarding Current Queue Until Proper Addition")
                # discard this queue 
                continue
            print(f"Please assign Queue{i+1} 's weight")
            weight = int(input(">"))
            q = PacketQueue(weight,q_packets)
            self.queues_vlist.extend(q.calculate_virtual_finish_time())
            self.pqueues.append(q)
        del q 
        del q_packets

    
    def assign_order(self):
        self.order_list = []
        self.queues_vlist = sorted(self.queues_vlist , key = lambda x : (x[0],x[1]))
        counter = 1
        for i in (self.queues_vlist):
            packet = self.pqueues[i[1]-1].get_packet(i[2])
            packet.set_order(counter)
            counter += 1
            self.order_list.append(packet)
        self.order_list.reverse()
        del self.queues_vlist
                

    def calculate_actual_finish_time(self):
        global_finish_time = 0
        while self.order_list:
            # because pop starts from last element which is technically the first packet now
            p = self.order_list.pop()
            if p.arrival_time >= global_finish_time:
                p.set_actual_finish_time(p.arrival_time)
            else:
                p.set_actual_finish_time(global_finish_time)
            global_finish_time = p.actual_finish_time
        del self.order_list
                
    
    def print_WFQ(self):
        print(" N |Arv| L |W.L| VFT | AFT | order")
        for packet_queue in self.pqueues:
            packet_queue.print_queue()
        
            
    

In [61]:
n = int(input("Enter number of Queues : "))
wfq = WFQ(n)
wfq.assign_order()
wfq.calculate_actual_finish_time()
wfq.print_WFQ()

Welcome to WFQ registry ...
Please enter required Packet to the format given below
Format: Packet_Name | Arrival Time | Length (-1 to exit)
Registry of Queue 1 packets
Please assign Queue1 's weight
Registry of Queue 2 packets
Please assign Queue2 's weight
Registry of Queue 3 packets
Please assign Queue3 's weight
 N |Arv| L |W.L| VFT | AFT | order
-------------Q1--------------
 A | 0 | 2 | 1 | 1 | 2 | 1
-------------Q2--------------
 A | 0 | 3 | 3 | 3 | 11 | 3
-------------Q3--------------
 A | 0 | 6 | 2 | 2 | 8 | 2


## handeled corner cases 
- Invalid Packet Format -> Discards the queue in hand 
- Non divisable weight -> Discards the queue in hand
- Duplicate Packet Naming in the same queue -> Discards the queue in hand
- Empty Queue -> Doesn't Affect Calculations 
- Duplicate Packets in global queue -> Doesn't Affect Calculations
- Having Same arrival time in queue -> Sort on Letter's ASCII value
- Having Same Virtual Finish time -> Sort on queues Number 